In [37]:
import pandas as pd
from sportsreference.ncaab.roster import Player

In [7]:
from sportsreference.ncaab.roster import Player

In [49]:
pd.set_option('display.max_columns', 999)

In [12]:
#variables to test
# zion = Player("zion-williamson-1")
# scoonie = Player("scoonie-penn-1")
# print(scoonie.height)
# df = zion('Career').dataframe
# df

In [51]:
#test string replacement
dude = "R.J. Barrett"
dude_id = dude.replace(".", "").replace(" ", "-").lower()+suf
dude_id

'rj-barrett-1'

In [52]:
#create function to get player stats
def getStats(player):
    suffix = "-1"
    ids = player.replace(".", "").replace(" ", "-").lower()+suffix
    college = Player(ids)
    df = college.dataframe
    return df


In [53]:
getStats("Lonzo Ball")

,assist_percentage,assists,block_percentage,blocks,box_plus_minus,conference,defensive_box_plus_minus,defensive_rebound_percentage,defensive_rebounds,defensive_win_shares,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,games_played,games_started,height,minutes_played,offensive_box_plus_minus,offensive_rebound_percentage,offensive_rebounds,offensive_win_shares,personal_fouls,player_efficiency_rating,player_id,points,points_produced,position,steal_percentage,steals,team_abbreviation,three_point_attempt_rate,three_point_attempts,three_point_percentage,three_pointers,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_attempts,two_point_percentage,two_pointers,usage_percentage,weight,win_shares,win_shares_per_40_minutes
2016-17,31.4,274,2.1,28,12.2,pac-12,3.5,14.3,183,2.0,0.668,343,0.551,189,0.286,98,0.673,66,36,36,6-6,1263,8.7,3.2,33,4.8,65,24.7,lonzo-ball-1,524,608,Guard,2.8,66,ucla,0.566,194,0.412,80,9.3,216,0.673,18.6,89,149,0.732,109,18.1,190,6.8,0.214
Career,31.4,274,2.1,28,12.2,,3.5,14.3,183,2.0,0.668,343,0.551,189,0.286,98,0.673,66,36,36,6-6,1263,8.7,3.2,33,4.8,65,24.7,lonzo-ball-1,524,608,Guard,2.8,66,ucla,0.566,194,0.412,80,9.3,216,0.673,18.6,89,149,0.732,109,18.1,190,6.8,0.214


In [ ]:
# d = {'col1': [1, 2], 'col2': [3, 4]}
# df = pd.DataFrame(data=d)

In [54]:
d = {'name': ['LeBron', 'Booker', 'KP', 'Boban', 'Scrub'], 'Yrs': [16, 5, 4, 8, 1], 'Year': [2002, 2004, 2017,2015,2014], 
    'm/game': [30, 22, 24, 9, 3], 'ws/48': [.20, .13, .09, .15, 0.11]}
df_test = pd.DataFrame(data=d)

In [55]:
df_test

,name,Yrs,Year,m/game,ws/48
0,LeBron,16,2002,30,0.20
1,Booker,5,2004,22,0.13
2,KP,4,2017,24,0.09
3,Boban,8,2015,9,0.15
4,Scrub,1,2014,3,0.11


In [58]:
df_test['type'] = ''
df_test
    

,name,Yrs,Year,m/game,ws/48,type
0,LeBron,16,2002,30,0.20,
1,Booker,5,2004,22,0.13,
2,KP,4,2017,24,0.09,
3,Boban,8,2015,9,0.15,
4,Scrub,1,2014,3,0.11,


In [59]:
x = 12

if x<20 and x>10:
    print('nice')
else:
    print('boo')

nice


In [72]:
for index, row in df_test.iterrows():
    if row['Year'] < 2018 and row['Yrs'] <= 2:
        row['type'] = "Bust"
    elif row['Year'] == 2018 and row['Yrs'] < 2:
        row['type'] = "Bust"
    elif row['Year'] == 2019 and row['Yrs'] < 1:
        row['type'] = "Bust"
    elif row['m/game'] > 15 and row['ws/48'] > .15:
        row['type'] = "Star"
    elif row['m/game'] > 15 and row['ws/48'] < .15 and row['ws/48'] >= .1:
        row['type'] = "Above Avg"
    else:
        row['type'] = "Below Avg"

    

        

In [79]:
for i in range(len(df_test)) : 
    if df_test.iloc[i,2] < 2018 and df_test.iloc[i,1] <= 2:
        df_test.iloc[i,5] = "Bust"
    elif df_test.iloc[i,2] == 2018 and df_test.iloc[i,1] < 2:
        df_test.iloc[i,5] = "Bust"
    elif df_test.iloc[i,2] == 2019 and df_test.iloc[i,1] < 1:
        df_test.iloc[i,5] = "Bust"
    elif df_test.iloc[i,3] > 15 and df_test.iloc[i,4] > .15:
        df_test.iloc[i,5] = "Star"
    elif df_test.iloc[i,3] > 15 and df_test.iloc[i,4] < .15 and df_test.iloc[i,4] >= .1:
        df_test.iloc[i,5] = "Above Avg"
    else:
        df_test.iloc[i,5] = "Below Avg"

In [77]:
df_test

,name,Yrs,Year,m/game,ws/48,type
0,LeBron,16,2002,30,0.20,Star
1,Booker,5,2004,22,0.13,Above Avg
2,KP,4,2017,24,0.09,Below Avg
3,Boban,8,2015,9,0.15,Below Avg
4,Scrub,1,2014,3,0.11,Bust
